<a href="https://colab.research.google.com/github/EsCODEbar98/SLNN/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - Cats vs. Dogs

Unzip data

In [ ]:
!unzip train.zip -d train/
!unzip test.zip -d test/

Code to initiliaze Tensorflow 2.0 in Colab

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
%load_ext tensorboard
import datetime
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

**[TODO] Create a data loader function that returns a tuple with a tf.float32 tensor for the image and a label. Images must be resized to 128x128.**
**N.B.: filenames are formatted as class.number.jpg**

In [ ]:
def loader(filename):
  label=tf.strings.split(filename,'.')[-3]
  label=int(tf.strings.split(label,'/')[-1])
  image= tf.io.read_file(filename)
  image=tf.image.decode_jpeg(image)
  image=tf.image.convert_image_dtype(image,tf.float32)
  image= tf.image.resize(image,[128,128])
  return image,label



**[TODO] Create a tf.Dataset, map the loader function and prepare a batch object for training**

In [ ]:
img_list_train=tf.data.Dataset.list_files('/content/train/*.jpg')
img_dataset_train=img_list_train.map(loader).batch(16)


**Prepare Keras callback for Tensorboard**

In [ ]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
%tensorboard --logdir logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq='batch')

**[TODO] Import the MobileNetV2 model trained on ImageNet without the final layer**

In [ ]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(128,128,3),include_top=False,weights='imagenet')
base_model.trainable= False

**[TODO] Add a final classification layer for 2 classes and create the final Keras model**

In [ ]:
inputs=base_model.input
x= base_model(inputs)
x= tf.keras.layers.Flatten()(x)
output= tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs, output)

**[TODO] Compile the Keras model: specify the optimization algorithm, the loss function and the test metric**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

**[TODO] Train the Keras model**

In [ ]:
model.fit(img_dataset_train, epochs=3 ,callbacks=[tensorboard_callback])

**[TODO] Print model summary**

In [ ]:
model.summary()

**[TODO] Test the Keras model by computing the accuracy the whole test set**

In [ ]:
img_list_test=tf.data.Dataset.list_files('/content/test/*.jpg')
img_dataset_test=img_list_test.map(loader).batch(32)
model.evaluate(img_dataset_test)

**[TODO] Load Test image 'test/0.1083.jpg', visualize it and check the network prediction**

In [ ]:
img=img_list_test[47,:,:]
img=img[np.newaxis,:,:]
y_hat=model.predict(img)
plt.imshow(img[0,:,:,0])
print('Predicted digit: %d'% np.argmax(y_hat))